# **Análisis del mercado literario**

### **Tema**
En los últimos años, debido a los avances tecnológicos, el mercado literario está experimentando un cambio en su modelo de negocio. En los últimos años han aparecido 

nuevos formatos que están cambiando la forma de consumo de libros. 
Con este EDA se pretende analizar cómo está evolucionando la industria editorial y sus principales características.

Para ello, se han planteado las siguientes hipótesis:
##### Hipótesis principal
* El número de publicaciones anuales ha aumentado

##### Otras hipótesis

* La publicación de libros por categoría varía según el formato de lectura
* No hay correlación entre el número de reseñas y la puntuación media de los libros.

### **Obtención de los datos**

### Datasets y fuentes alternativas de datos

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import random

**Fuente 1:** Dataset de libros en kindle.com con más de 130 mil entradas. 

In [ ]:
df_kindle = pd.read_csv("../data/kindle_data-v2.csv")
fuente_1 = "https://www.kaggle.com/datasets/asaniczka/amazon-kindle-books-dataset-2023-130k-books"

**Fuente 2:** Dataset de audiolibros en audible.com con el top 100 de 24 categorías. 

In [ ]:
df_t100aud = pd.read_csv("../data/Audible_Top_100_Final.csv", encoding='windows-1252')
fuente_2 = "https://www.kaggle.com/datasets/ntsmith/audible-top-100-best-selling-all-categories"

**Fuente 3:** Dataset que incluye el top 100 de libros vendidos desde 2009 a 2021

In [ ]:
df_t100am = pd.read_excel("../data/BBDD_EDA/Amazon_top100.xlsx")
fuente_3 = "https://www.kaggle.com/datasets/abdulhamidadavize/top-100-best-selling-books-on-amazon-20092021"

**Fuente 4:** Dataset que contiene más de 100 mil referncias obtenidas de la página web de la librería americana Wonder Book. 

In [ ]:
bookdataset = pd.read_csv("../data/BBDD_EDA/BooksDatasetClean.csv")
fuente_4 = "https://www.kaggle.com/datasets/elvinrustam/books-dataset"

In [ ]:
### Por el momento sin utilizar los siguientes dataframes

#Puede que sí nos valga
df_2_best_selling = pd.DataFrame(pd.read_csv("../data/best-selling-books.csv"))
fuente_2 = "https://www.kaggle.com/datasets/drahulsingh/best-selling-books"


df_5_abs_sec_esp = pd.DataFrame(pd.read_csv("../data/abs_sect_editorial_esp.csv"))
fuente_5 = "https://www.federacioneditores.org/datos-estadisticos-2020.php"

df_6_dist_cifneg_gen = pd.DataFrame(pd.read_csv("../data/dist_cif_neg_genero_sect_editorial_esp.csv"))
fuente_6 = "https://www.federacioneditores.org/datos-estadisticos-2020.php"

# esta está bie, pero hay libros repetidos debido a que tienen diferente formato, quizás tampoco es la mejor, porque no te dice a qué formato pertenece cada uno
df_7_bestsell_amazon = pd.DataFrame(pd.read_csv("../data/bestsellers_with_categories_2022_03_27.csv"))
fuente_7 = "https://www.kaggle.com/datasets/chriskachmar/amazon-top-50-bestselling-books-2009-2022?select=bestsellers_with_categories_2022_03_27.csv"

# bbdd de Netflix para ver si se pudiera rastrear algo
df_8_imdb = pd.DataFrame(pd.read_csv("../data/Netflix_data.csv"))
fuente_8 = "https://www.kaggle.com/datasets/abdulszz/netflix-data"

# 20 adaptaciones de libros a pantalla
df_9_adaptations = pd.DataFrame(pd.read_csv("../data/bookfilm_adapt.csv"))
fuente_9 = "https://ew.com/best-book-to-screen-adaptations-of-all-time-8685833"

### Tratamiento de datos
En lineas generales, a todos los datasets se les ha realizado una limpieza, convirtiendo todas las strings a minúscula, eliminando espacios no necesarios y sustituyendo espacios en las etiquetas de columna por "_" para que sean más accesibles. También en aquellos casos necesarios, se sustituyó el texto "nan" por np.nan para que no afectaran al análisis y se eliminaron filas sin títulos de libros en caso de que existiera alguna.

Independientemente, al trabajar con varios datasets, he optado por hacerlo en libros distintos, para tener un mejor control de cada dataframe:

#### [Wonder Book]("../notebooks/kindle.ipynb")
En este dataset, a parte de realizar la limpieza a los datos ya comentada, también hubo que hacer una serie de ajustes ya que la columna con los datos de los autores incluía en todos ellos un "by" junto al nombre del autor. Ese "by" se sustituyo por "" (vacío). Después aquellos títulos que quedaron sin autor se sustituyó por **"unknown"** ya que puede haber libros sin autor específico.
Después, se eliminaron las filas con **títulos duplicados**. En este caso tomé la decisión de que me quedaría con los títulos que en caso de estar duplicados, tuvieran un autor no desconocido y cuyo año de publicación fuera el más antiguo.
Por otro lado, el dataset tenía una única columna de categoría donde incluía varios datos, por lo que decidí dividirla en diferentes columnas.
Tras ello, comprobé la primera columna de categoría que se había generado. Me llamó la atención que la que más valores tenía era *"ficción"*, bastante genérica, por lo que decidí crear una nueva columna denominada **"category_resumen_1"** en la que apliqué varios cambios.
 1. Primero senté la base de los valores, que era que si el género de ese título era ficción, que devolviera la segunda categoría de género, si no, que se quedara con el género de "category_1".
 2. Una vez aplicado ese cambio, tomando como referencia las categorías de kindle, apliqué un cambio de denominación a aquellas categorías que tenían un número elevado de referencias, para que fuera lo más comparable posible el dato con los otros dataset.

Una vez hecho esto, decidí crear otra columna con el **top 10** de géneros, para poder graficarlos mejor. Así que a todo aquel género que no estuviera en la lista del top 10, se le renombraría como "other", excepto los nulos, que se dejarían como tal.

También decidí crear una nueva columna de bins con las décadas de publicación para poder graficar mejor el año de publicación, ya que el rango era muy amplio. Decidí que los bins serían todas las décadas desde los años 50 hasta la del 2010. Todos los libros publicados anteriormente a 1950 están en el bin *"before 1950"* y todos los publicados a partir de 2020 están en *"after 2019"*.

Tras ello eliminé todas las columnas del dataframe que no necesitaba para posteriores análisis.


#### [Kindle]("../notebooks/kindle.ipynb")
Este dataset de Kindle incluye más de 130 mil referencias. Este al igual que el anterior se ha limpiado, se han creado las columnas de bins de **décadas** para los años y la columna del **top 10**. 
He decidido eliminar también todas las filas de título duplicado siguiendo los mismos criterios que en el dataset anterior, y también he eliminado todas las filas de las que no había año de publicación. 
También he agregado una columna de ranking según el número de reviews, que podríamos considerar como **nº de lecturas** por categoría.

#### [Audible]("../notebooks/kindle.ipynb")
En este dataset se muestra el top 100 de audiolibros vendidos por categoría. Como en los casos anteriores se ha limpiado, y se le ha añadido la columna del **top 10** desde el último dataframe de kindle.
Con este dataset se busca comprobar la evolución de los audiolibros.

#### [Amazon]("../notebooks/kindle.ipynb")
En este dataset se muestra el top 100 de libros vendidos en Amazon desde 2009 a 2021. Como en los casos anteriores se ha limpiado, y se le ha añadido la columna del **top 10** desde el último dataframe de kindle.
Con este dataset por un lado se pretende ver la evolución de las ventas según el tipo de cubierta del libro.



El siguiente es el código utilizado para limpiar los datasets. 

In [ ]:
def clean_dataset(df):
    df.drop(df[df["title"].isnull()].index, inplace=True)
    #df.drop(columns=['Unnamed: 0'], inplace=True)
    df = df.dropna(how="all", axis= 1)
    for column in df.columns:
        #print("Va por: ", column)
        if df[column].dtype == object:
            #print("tipo columna: ", df[column].dtype)
            df[column] = df[column].str.lower()
            df[column] = df[column].astype(str).str.strip()
            df.replace({'nan': np.nan}, inplace=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.drop(df[df["title"].isnull()].index, inplace=True)
    df["author"] = np.where(df["author"] == "", "unknown", df["author"])  
    return df

df_kd = clean_dataset(df)  

Con el código siguiente se ha incluido en los dataframe la columna de rango de años para trabajar con los datos en décadas en vez de año a año.

In [ ]:
# para crear columna de año a partir de la de fecha de publicacion
df["published_year"] = df["publisheddate"].str.split("-").str[0]
df["published_year"] = df["published_year"].astype("Int64")
# Nueva columna de rangos de décadas. En kindle va desde los 2000
bins = [-float('inf'), 1989, 1994, 1999, 2004, 2009, 2014, 2019, float('inf')]
labels = ['before 1990', '1990-1994', '1995-1999', '2000-2004', '2005-2009', '2010-2014', '2015-2019', 'after 2019']

df_bd['year_range'] = pd.cut(df_bd['publish_date_(year)'], bins=bins, labels=labels)

Y este código se ha utilizado para elimianr los duplicados de título. Se ha priorizado el quedarnos con aquellos títulos que tenían un autor asociado y con el año de publicación más antiguo.

In [ ]:
# para eliminar duplicados
df_duplicated = df_kd[df_kd["title"].duplicated()].sort_values(by= "author", ascending= True)
df_kd.drop(df_duplicated[df_duplicated["author"] == "Unknown"].index, inplace= True)
# para eliminar duplicados quedándonos con el primer año de publicación del mismo título
df_kd.sort_values(by="published_year", inplace=True)
df_kd.drop_duplicates("title", inplace=True)

El código siguiente se ha utilizado para generar la variable de top 10 de categorías. Ordenando las categorías según su conteo y filtrandola por las 10 mayores. El resto de categorías si existían han pasado a ser "others". 

In [ ]:
# Para generar variable top_10 de categorías
top_10_categories = df_kd["category_name"].value_counts().nlargest(10).index
df_kd['category_top_10'] =  np.nan
df_kd['category_top_10'] = np.where(df_kd['category_name'].isna(), np.nan, np.where(df_kd['category_name'].isin(top_10_categories), df_kd['category_name'], 'others'))

Para el dataset de kindle, como se comenta más arriba, también he creado un ranking por número de reviews de cada categoría

In [ ]:
# creación de un ranking según el número de reviews por categoría.
categories = df_kd["category_top_10"].unique()
for categorie in categories:
    #print(categorie)
    df_categorie = df_kd[df_kd["category_top_10"] == categorie].copy()
    df_categorie = df_categorie.sort_values(by= ["reviews"], ascending= False)
    df_categorie["ranking_" + str(categorie)] = range(1, len(df_categorie) + 1)
    #print(len(df_categorie))
    df_kd = pd.merge(df_kd,df_categorie, how="left")

## HIPÓTESIS 1: *El número de publicaciones anuales ha aumentado*

Para esta hipótesis vamos a tener en cuenta el dataset de Wonder Book y el dataset de Kindle.

En cada uno se va a graficar un gráfico de barras que muestra el número de libros publicados por década.
Para que el gráfico salga ordenado, primero se vuelven a ordenar todos los valores por año de publicación.

Luego con un groupby he obtneido los datos a graficar, es decir el número de libros publicados por década, tanto en Kindle como en Wonderbook y el resultado lo he vuelto a guardar como un dataframe para graficarlo en un gráfico de barras con seaborn.



In [ ]:
# Para graficar los datos referidos a publicaciones totales por década.
titles_decade_count = df_kd_h1.groupby("year_range")["title"].count().reset_index(name='title_count')
sns.barplot(titles_decade_count, x = "year_range", y = "title_count")
plt.xticks(rotation= 45)
plt.title('Nº LIBROS PUBLICADOS POR AÑO EN PAPEL', fontsize=18)
plt.xlabel('Décadas', fontsize=14)
plt.ylabel('Nº de libros', fontsize=14);

## HIPÓTESIS 2: *La publicación de libros por categoría varía según el formato de lectura*

En este caso, lo que se busca es ver si hay en la distribución de la publicación de libros dependiendo del formato de lectura.

para ello se ha hecho un groupby por rango de años y el top10 de categoría y se ha hecho un conteo del nº de títulos que cumplen esas condiciones. Después se ha pasado a una pivot table y posteriormente se han calculado las frecuencias para graficarlo de forma que el valor representado sea la distribución en porcentaje de los géneros publicados por rango de años.

In [ ]:
generos_decada = df_kd_h1.groupby(["year_range","category_top_10"])["title"].size().reset_index(name='counts')
generos_decada_piv = generos_decada.pivot(index='year_range', columns='category_top_10', values='counts')
#para ordenar los valores
category_totals = generos_decada_piv.sum().sort_values(ascending=False)
generos_decada_piv  = generos_decada_piv [category_totals.index]
# para calcular las frecuencias
totales_decada = generos_decada_piv.sum(axis=1)
generos_decada_per = generos_decada_piv.div(totales_decada, axis =0) * 100

# gráfico de la distribución
generos_decada_per.plot(kind="bar", stacked=True, figsize=(10,6), colormap="tab20")
plt.title('Distribución de Categorías por Década en formato ebook')
plt.ylabel('Distribución(%)')
plt.xlabel('Década')
plt.xticks(rotation=20)
plt.legend(title='Categorías',bbox_to_anchor=[1, 0.75], loc='upper left')
plt.tight_layout()
plt.show()

# bases de cada rango de fechas
generos_decada_piv.sum(axis=1)

## HIPÓTESIS 3: *No hay correlación entre el número de reseñas y la puntuación media de los libros.*

Se quiere comprobar si existe algún tipo de relación entre las valoraciones recibidas y la cantidad de reseñas recibidas por categoría y formato

Para ello se ha realizado un scatterplot poniendo en el eje x la puntuación de las valoraciones, en el eje y el número de reseñas recibidas y el filtro de color por category_top_10.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=filtered_audible, x='overall_rating_out_of_5', y='ratings', hue='category_top_10', palette='viridis', s=100)

# Añadir etiquetas y título
plt.title('Ratings vs. Overall Rating (agrupado por Category Top 10)', fontsize=14)
plt.xlabel('Overall Rating (out of 5)')
plt.ylabel('Number of Ratings')

# Mostrar el gráfico
plt.show()